In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
#ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data=Dataset.load_from_file('C:\\Users\\bumbk\\capstone_gbt\\ratings_noh.csv',reader=reader)

In [5]:
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5,5))

data_folds = DatasetAutoFolds(ratings_file='C:\\Users\\bumbk\\capstone_gbt\\ratings_noh.csv',reader=reader)

trainset=data_folds.build_full_trainset()

In [16]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [21]:
movies = pd.read_csv('C:\\Users\\bumbk\\Downloads\\ml-25m\\ml-25m\\movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
41       42  Dead Presidents (1995)  Action|Crime|Drama


In [24]:
len(movies)

62423

In [22]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 2.98   {'was_impossible': False}


In [23]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
         '전체 영화 수:', len(total_movies))
    
    return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 178 추천 대상 영화 수: 62245 전체 영화 수: 62423


In [25]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top]
    
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [ (id, title, rating) for id, title, rating in \
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds


unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top=10)

print('####Top-10 추천 가구 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화수: 178 추천 대상 영화 수: 62245 전체 영화 수: 62423
####Top-10 추천 영화 리스트 ####
Anastasia (1997) : 4.890874405552288
Titanic (1997) : 4.886536362862591
Star Trek: Insurrection (1998) : 4.883295018488769
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) : 4.8324047076714
Harry Potter and the Chamber of Secrets (2002) : 4.798345905139476
Harry Potter and the Goblet of Fire (2005) : 4.795400500876132
Harry Potter and the Order of the Phoenix (2007) : 4.776272799894801
Fireproof (2008) : 4.774664488268597
Harry Potter and the Half-Blood Prince (2009) : 4.772378429229023
Harry Potter and the Deathly Hallows: Part 2 (2011) : 4.770145179054808
